In [80]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt

# Idea to generate a random SNF to compare it to others in the category.
import random

In [81]:
url="https://data.cms.gov/provider-data/api/1/metastore/schemas/dataset/items/4pq5-n9py?show-reference-ids=false"

In [82]:
print(requests.get(url))

<Response [200]>


In [83]:
print(requests.get(url).json())

{'accessLevel': 'public', 'landingPage': 'https://data.cms.gov/provider-data/dataset/4pq5-n9py', 'bureauCode': ['009:38'], 'issued': '2023-08-01', '@type': 'dcat:Dataset', 'modified': '2024-04-01', 'released': '2024-04-24', 'keyword': [{'identifier': 'd03021cc-311c-5181-8eb6-7ccf7e76fc53', 'data': 'General Information'}, {'identifier': '203355e5-2454-505d-b356-08de15938ab9', 'data': 'Address'}, {'identifier': 'c59e48c7-3cc5-5a09-adec-a49493db72f6', 'data': 'Location'}, {'identifier': '70a212e5-bf1b-5cad-9ecd-150dab2bacf9', 'data': 'Ratings'}, {'identifier': '107b8826-e90b-5db3-ac30-ecdb99c73db6', 'data': 'Beds'}, {'identifier': '5692b1a6-1c82-55c4-9698-c7441733b985', 'data': 'Quality'}, {'identifier': 'a6f7b7c6-d26e-5f59-8a5b-c151bacb9907', 'data': 'Staffing'}, {'identifier': '35df7004-e437-5a8a-a96b-1bda5456ccca', 'data': 'Penalties'}], 'contactPoint': {'@type': 'vcard:Contact', 'fn': 'Nursing homes including rehab services', 'hasEmail': 'mailto:BetterCare@cms.hhs.gov'}, 'publisher': 

In [84]:
response = requests.get(url).json()
print(json.dumps(response, indent=4, sort_keys=True))

{
    "%modified": "2024-04-12T16:05:17-0400",
    "@type": "dcat:Dataset",
    "accessLevel": "public",
    "bureauCode": [
        "009:38"
    ],
    "contactPoint": {
        "@type": "vcard:Contact",
        "fn": "Nursing homes including rehab services",
        "hasEmail": "mailto:BetterCare@cms.hhs.gov"
    },
    "description": "General information on currently active nursing homes, including number of certified beds, quality measure scores, staffing and other information used in the Five-Star Rating System. Data are presented as one row per nursing home.",
    "distribution": [
        {
            "data": {
                "%Ref:downloadURL": [
                    {
                        "data": {
                            "checksum": null,
                            "filePath": "s3://pdc-s3-nhc-data/NH_ProviderInfo_Apr2024.csv",
                            "identifier": "5eba4f43478aeb4d0ab8d071e1e4655d",
                            "mimeType": "text/csv",
           

In [92]:
response = requests.get(url).json()
print(json.dumps(response["distribution"][0]["data"]["downloadURL"], indent=4, sort_keys=True))

"https://data.cms.gov/provider-data/sites/default/files/resources/5eba4f43478aeb4d0ab8d071e1e4655d_1712952317/NH_ProviderInfo_Apr2024.csv"


In [93]:
nursinghomedataURL = json.dumps(response["distribution"][0]["data"]['downloadURL'], indent=4, sort_keys=True)
print(nursinghomedataURL)

"https://data.cms.gov/provider-data/sites/default/files/resources/5eba4f43478aeb4d0ab8d071e1e4655d_1712952317/NH_ProviderInfo_Apr2024.csv"


In [94]:
cleaned_string = nursinghomedataURL.replace('"', ' ').strip()

In [95]:
cleaned_string

'https://data.cms.gov/provider-data/sites/default/files/resources/5eba4f43478aeb4d0ab8d071e1e4655d_1712952317/NH_ProviderInfo_Apr2024.csv'

In [96]:
# pd.read_csv(nursinghomedataURL).sample(5)
nursinghomedata=pd.read_csv(cleaned_string)
nursinghomedata.sample(5)

,CMS Certification Number (CCN),Provider Name,Provider Address,City/Town,State,ZIP Code,Telephone Number,Provider SSA County Code,County/Parish,Ownership Type,...,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Latitude,Longitude,Geocoding Footnote,Processing Date
4326,165438,STACYVILLE COMMUNITY NURSING HOME,413 SOUTH BROAD STREET,STACYVILLE,IA,50476,6417102215,650,Mitchell,Non profit - Corporation,...,NaN,5,6619.91,0,5,"413 SOUTH BROAD STREET,STACYVILLE,IA,50476",43.4325,-92.782,NaN,2024-04-01
7286,265693,"REHABILITATION CENTER OF INDEPENDENCE, THE",1800 S SWOPE DRIVE,INDEPENDENCE,MO,64057,8162572566,470,Jackson,For profit - Limited Liability company,...,1.0,3,154289.01,0,3,"1800 S SWOPE DRIVE,INDEPENDENCE,MO,64057",39.0853,-94.369,NaN,2024-04-01
6267,235515,LAKEVIEW MANOR HEALTHCARE CENTER,408 N FIFTH AVE,TAWAS CITY,MI,48763,9893622211,340,Iosco,For profit - Limited Liability company,...,NaN,0,0.00,0,0,"408 N FIFTH AVE,TAWAS CITY,MI,48763",44.2754,-83.522,NaN,2024-04-01
10042,366199,COUNTRY LANE GARDENS REHAB & NURSING CTR,7820 PLEASANTVILLE ROAD,PLEASANTVILLE,OH,43148,7405367381,230,Fairfield,For profit - Corporation,...,7.0,4,55250.00,0,4,"7820 PLEASANTVILLE ROAD,PLEASANTVILLE,OH,43148",39.8061,-82.446,NaN,2024-04-01
11368,396144,ACCELERATE SKILLED NURSING AND REHABILITATION ...,501 THOMAS JONES WAY,EXTON,PA,19341,6104238600,210,Chester,For profit - Corporation,...,NaN,0,0.00,0,0,"501 THOMAS JONES WAY,EXTON,PA,19341",40.0222,-75.653,NaN,2024-04-01


In [97]:
# nursinghomedataREDUCED=nursinghomedata.drop(columns=['CMS Certification Number (CCN)', 'Provider SSA County Code', 'Geocoding Footnote', 'Processing Date'])
nursinghomedataREDUCED=nursinghomedata[[
        'Provider Name', 
        'City/Town',
        'State', 
        'Ownership Type', 
        'Number of Certified Beds', 
        'Average Number of Residents per Day', 
        'Number of Facility Reported Incidents', 
        'Number of Substantiated Complaints', 
        'Number of Citations from Infection Control Inspections', 
        'Number of Fines', 
        'Total Amount of Fines in Dollars',
        'Abuse Icon']]
    

nursinghomedataREDUCED.sample(25)

,Provider Name,City/Town,State,Ownership Type,Number of Certified Beds,Average Number of Residents per Day,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Citations from Infection Control Inspections,Number of Fines,Total Amount of Fines in Dollars,Abuse Icon
10022,CUMBERLAND POINTE CARE CENTER,ST CLAIRSVILLE,OH,For profit - Corporation,75,61.6,0,9,0.0,3,29650.00,N
6951,BOLIVAR MEDICAL CENTER LTC,CLEVELAND,MS,For profit - Corporation,35,24.4,3,0,NaN,1,650.00,N
3938,SOUTHFIELD VILLAGE,SOUTH BEND,IN,Government - City/county,60,49.5,0,6,NaN,1,6922.50,N
8802,THE OSBORN,RYE,NY,Non profit - Corporation,84,74.0,0,3,0.0,0,0.00,N
10792,COLE PLACE,COUDERSPORT,PA,Non profit - Corporation,44,25.7,1,0,NaN,2,1975.47,N
12395,SPRINGFIELD HEALTH & REHAB,SPRINGFIELD,VT,For profit - Corporation,102,69.5,4,11,0.0,1,7432.75,N
8734,ELLIS RESIDENTIAL & REHABILITATION CENTER,SCHENECTADY,NY,Non profit - Corporation,82,70.1,0,0,NaN,2,3122.75,N
14431,LAMPSTAND NURSING AND REHABILITATION,BRYAN,TX,For profit - Corporation,140,88.3,3,3,2.0,4,144337.89,N
7302,DELMAR GARDENS OF MERAMEC VALLEY,FENTON,MO,For profit - Corporation,190,159.1,2,41,0.0,2,1625.00,N
5638,MARYLAND BAPTIST AGED HOME,BALTIMORE,MD,Non profit - Church related,29,22.6,0,0,0.0,3,6962.74,N


In [98]:
nursinghomedata.columns

Index(['CMS Certification Number (CCN)', 'Provider Name', 'Provider Address',
       'City/Town', 'State', 'ZIP Code', 'Telephone Number',
       'Provider SSA County Code', 'County/Parish', 'Ownership Type',
       'Number of Certified Beds', 'Average Number of Residents per Day',
       'Average Number of Residents per Day Footnote', 'Provider Type',
       'Provider Resides in Hospital', 'Legal Business Name',
       'Date First Approved to Provide Medicare and Medicaid Services',
       'Affiliated Entity Name', 'Affiliated Entity ID',
       'Continuing Care Retirement Community', 'Special Focus Status',
       'Abuse Icon', 'Most Recent Health Inspection More Than 2 Years Ago',
       'Provider Changed Ownership in Last 12 Months',
       'With a Resident and Family Council',
       'Automatic Sprinkler Systems in All Required Areas', 'Overall Rating',
       'Overall Rating Footnote', 'Health Inspection Rating',
       'Health Inspection Rating Footnote', 'QM Rating', 'QM Rating